<a href="https://colab.research.google.com/github/SprihaT/XOR-Logic-Gate/blob/main/BinaryXOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np

# Slope function (Derivative function)
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])

    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x + delta_x)

        x[idx] = tmp_val - delta_x
        fx2 = f(x) # f(x - delta_x)
        grad[idx] = (fx1 - fx2) / (2 * delta_x)

        x[idx] = tmp_val # Restore value
        it.iternext()

    return grad

# Sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class LogiGate:
    def __init__(self, gate_name, xdata, tdata):
        self.name = gate_name

        # Creating Input training data
        self.__xdata = xdata.reshape(4, 2) # 4 samples of 2 inputs
        self.__tdata = tdata.reshape(4, 1) # 4 samples of 1 output

        # Hidden layer unit
        self.__W2 = np.random.rand(2, 6) # Weight W2, 2 x 6 matrix
        self.__b2 = np.random.rand(6)

        # Creating output layer unit: Weight W3, Bias b3
        self.__W3 = np.random.rand(6, 1)
        self.__b3 = np.random.rand(1)

        # Learning rate
        self.__learning_rate = 1e-2

        print(self.name + " object is created")

    def feed_forward(self):
        small_number = 1e-7 # No zero in log function

        z2 = np.dot(self.__xdata, self.__W2) + self.__b2 # Output of Hidden Layer
        a2 = sigmoid(z2)

        z3 = np.dot(a2, self.__W3) + self.__b3 # Output of Output layer
        y = sigmoid(z3)

        # Cross-entropy
        return -np.sum(self.__tdata * np.log(y + small_number) +
                       (1 - self.__tdata) * np.log(1 - y + small_number))

    def loss_val(self):
        delta = 1e-7 # No zero in log function

        z2 = np.dot(self.__xdata, self.__W2) + self.__b2 # Output of Hidden Layer
        a2 = sigmoid(z2)

        z3 = np.dot(a2, self.__W3) + self.__b3 # Output of Output Layer
        y = sigmoid(z3)

        # Cross-entropy
        return -np.sum(self.__tdata * np.log(y + delta) +
                       (1 - self.__tdata) * np.log(1 - y + delta))

    # Minimize Cost function
    def train(self):
        f = lambda x: self.feed_forward()

        print("Initial loss value =", self.loss_val())

        for step in range(10001):
            self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
            self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
            self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
            self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)

            if step % 400 == 0:
                print("step =", step, "loss value =", self.loss_val())

    # Test the output
    def predict(self, xdata):
        z2 = np.dot(xdata, self.__W2) + self.__b2 # Output of Hidden Layer
        a2 = sigmoid(z2)

        z3 = np.dot(a2, self.__W3) + self.__b3 # Output of Output Layer
        y = sigmoid(z3)

        if y > 0.5:
            result = 1 # True
        else:
            result = 0 # False

        return result


# XOR() Function and its training
xdata = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
tdata = np.array([[0], [1], [1], [0]]) # Changed the tdata for XOR gate

xor_obj = LogiGate("XOR", xdata, tdata) # Changed the object name to xor_obj
xor_obj.train() # Training the XOR gate


XOR object is created
Initial loss value = 8.223846249367611
step = 0 loss value = 8.069968652327452
step = 400 loss value = 2.772997826261382
step = 800 loss value = 2.768040544034805
step = 1200 loss value = 2.76326259556717
step = 1600 loss value = 2.7580467575604932
step = 2000 loss value = 2.7517617595498858
step = 2400 loss value = 2.7436489529905725
step = 2800 loss value = 2.7327183963713266
step = 3200 loss value = 2.717653839922858
step = 3600 loss value = 2.6967467464964585
step = 4000 loss value = 2.6678982920228407
step = 4400 loss value = 2.6287370721654004
step = 4800 loss value = 2.576901761694468
step = 5200 loss value = 2.5105391704990856
step = 5600 loss value = 2.4289672691630932
step = 6000 loss value = 2.3330831928902764
step = 6400 loss value = 2.224859130263712
step = 6800 loss value = 2.105994771360959
step = 7200 loss value = 1.9768307253597017
step = 7600 loss value = 1.8364883155996874
step = 8000 loss value = 1.6842714132896033
step = 8400 loss value = 1.52

In [29]:
import plotly.graph_objects as go
import numpy as np

# Generate input data for the graph
x_range = np.arange(0, 1, 0.05)
y_range = np.arange(0, 1, 0.05)
x, y = np.meshgrid(x_range, y_range)
z = np.zeros_like(x)

# Predict the output for each input combination using the trained xor_obj
for i in range(len(x_range)):
  for j in range(len(y_range)):
    input_data = np.array([[x[i, j], y[i, j]]])
    output = xor_obj.predict(input_data)
    z[i, j] = output

# Create the 3D surface graph
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])

# Customize the layout
fig.update_layout(title='XOR Gate Output',
                  scene=dict(
                      xaxis_title='X',
                      yaxis_title='Y',
                      zaxis_title='Z (Output)'))

# Show the graph
fig.show()

In [30]:
# Get the prediction for the input [1, 1]
input_data = np.array([[1, 1]])
output = xor_obj.predict(input_data)

print(f"Output for input [1, 1]: {output}")

Output for input [1, 1]: 0
